In [1]:
import numpy as np
import csv, os
from torch.utils.data import DataLoader
from dgl.data.utils import split_dataset
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy import stats

from src_chung.model import reactionMPNN, training, inference
from src_chung.dataset import GraphDataset
from src_chung.util import collate_reaction_graphs

/home/phuocchung123/anaconda3/envs/GIN_korea/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = GraphDataset('./data_chung/data_valid2.npz')
valid_loader = DataLoader(
        dataset=data,
        batch_size=32,
        shuffle=True,
        collate_fn=collate_reaction_graphs,
        drop_last=True,
    )

In [18]:
import dgl
for batch in valid_loader:
    a=batch[0]
    b=dgl.slice_batch(a,0)
    break

In [16]:
b

Graph(num_nodes=4, num_edges=6,
      ndata_schemes={'attr': Scheme(shape=(155,), dtype=torch.float32)}
      edata_schemes={'edge_attr': Scheme(shape=(8,), dtype=torch.float32)})

In [13]:
b=slice_batch(data,0)

NameError: name 'slice_batch' is not defined

In [3]:
node_dim = valid_loader.dataset.rmol_node_attr[0].shape[1]
edge_dim = valid_loader.dataset.rmol_edge_attr[0].shape[1]

In [4]:
for batch in valid_loader:
    # if batch[-1].size()[0] != 32:
    a=[b for b in batch[:23]]
    b=[b for b in batch[23:23+6]]
    break

/mnt/d/workspace/kaggle_recat/src_chung/util.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  labels = torch.stack([torch.argmax(y) for y in torch.Tensor(batchdata[-1])], axis=0)


In [5]:
from src_chung.model import reactionMPNN
import torch
de=reactionMPNN(node_dim,edge_dim)
x=torch.cat([de.mpnn(i) for i in a])
# for mol in a:
#     x.append(de.mpnn(mol))
# len(x)

/home/phuocchung123/anaconda3/envs/GIN_korea/lib/python3.11/site-packages/dgl/core.py:82: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  dgl_warning(


In [6]:
x.shape

torch.Size([1834, 300])

In [1]:
import torch

# Suppose you have the following list of tensors
list_of_tensors = [torch.tensor([1, 2, 3]), torch.tensor([4, 5, 6]), torch.tensor([7, 8, 9])]

# You can create a new tensor using torch.stack()
new_tensor = torch.stack(list_of_tensors)

print(new_tensor)


tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


In [11]:
a_tensor=torch.Tensor()
b_tensor=torch.cat((a_tensor,new_tensor))
b_tensor.shape

torch.Size([3, 3])

In [9]:
new_tensor[0:new_tensor[0,0]]

tensor([[1, 2, 3]])

In [8]:
import torch

# Your tensors
tensor1 = torch.tensor([[1, 2, 3],[4,5,6],[20,21,22]])
# tensor2 = torch.tensor([[10,11,12],[30,31,32]])
tensor3 = torch.tensor([])
print(tensor3.shape)
# Expand dimensions of tensor2 and tensor3
# tensor2 = tensor2.unsqueeze(0)
# tensor3 = tensor3.unsqueeze(0)
# print(tensor2.shape)

# Now you can concatenate them
result = torch.cat((tensor1, tensor3))

print(result)


torch.Size([0])
tensor([[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [20., 21., 22.]])


In [9]:
tensor2 = torch.tensor([[10,11,12],[30,31,32]])
result_2=torch.cat((tensor2,result))

In [15]:
x=torch.tensor([])

In [21]:
torch.sum(torch.tensor([[1,2,3],[4,5,6]]),0).unsqueeze(0)

tensor([[5, 7, 9]])

In [26]:
import torch
# Example of target with class indices
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.randn(3, 5).softmax(dim=1)
print(target)
output = loss(input, target)
output.backward()

tensor([[-0.3916,  0.2927, -0.6941,  0.3057, -0.3279],
        [-0.7313,  1.4835,  0.2858,  0.1565, -1.9621],
        [-0.4837, -0.5856,  0.8011,  1.2600, -0.1051]], requires_grad=True)
tensor([3, 4, 2])
tensor([[ 1.4492,  0.7545, -0.0275, -0.7063,  1.0340],
        [-0.5540, -0.2978,  0.4156,  0.8164, -1.0275],
        [-1.1896,  0.4765,  0.9827,  0.6630,  1.4780]], requires_grad=True)
tensor([[0.1577, 0.0583, 0.2158, 0.4900, 0.0782],
        [0.3218, 0.0671, 0.2131, 0.3022, 0.0957],
        [0.0972, 0.1144, 0.4426, 0.0993, 0.2465]])


In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, emb_dim, dim_kqv):
        super(AttentionHead, self).__init__()
        self.dim_kqv = dim_kqv
        
        self.wq = nn.Linear(emb_dim, dim_kqv)
        self.wk = nn.Linear(emb_dim, dim_kqv)        
        self.wv = nn.Linear(emb_dim, dim_kqv)
        
    def forward(self, q, k, v, mask):
        queries = self.wq(q)
        keys = self.wk(k)
        values = self.wv(v)
        
        score = queries.bmm(keys.transpose(1, 2))     
          
        score = torch.div(score, self.dim_kqv ** 0.5, rounding_mode='floor')
        
        if mask is not None:
            score = score.masked_fill(mask == 0, -1e9)
        
        softmax = F.softmax(score, dim = -1)

        return softmax.bmm(values)
    
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, emb_dim, dim_kqv):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList(
            [AttentionHead(emb_dim, dim_kqv) for _ in range(num_heads)]
        )
        
        self.w0 = nn.Linear(num_heads * dim_kqv, emb_dim)
        
    def forward(self, q, k, v, mask):
        attentions = [h(q, k, v, mask) for h in self.heads]
        attentions = torch.cat(attentions, dim = -1)
        out = self.w0(attentions)
        
        return out

class Residual(nn.Module):
    def __init__(self, sublayer, dimension, dropout):
        super(Residual, self).__init__()
        self.sublayer = sublayer
        self.norm = nn.LayerNorm(dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, *tensors):
        return self.dropout(self.norm(tensors[0] + self.sublayer(*tensors)))
    
class FeedForward(nn.Module):
    def __init__(self, emb_dim, ff_dim):
        super(FeedForward, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(emb_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, emb_dim)
        )
        
    def forward(self, residual_out):
        return self.network(residual_out)
    

class EncoderLayer(nn.Module):
    def __init__(self, emb_dim, num_heads, ff_dim, dropout):
        super(EncoderLayer, self).__init__()
        self.dim_kqv = emb_dim // num_heads
        
        assert (self.dim_kqv * num_heads == emb_dim), "Embedding size must be divisible by number of heads" 
        
        self.attention = Residual(
            MultiHeadAttention(num_heads, emb_dim, self.dim_kqv),
            dimension=emb_dim,
            dropout=dropout,
        )
    
        self.feed_forward = Residual(
            FeedForward(emb_dim, ff_dim),
            dimension=emb_dim,
            dropout=dropout,
        )
        
    def forward(self, src, mask):
        src = self.attention(src, src, src, mask)
        out = self.feed_forward(src)
        return out
    
class Encoder(nn.Module):
    def __init__(self, 
                 emb_dim, 
                 num_heads, 
                 ff_dim, 
                 num_layers, 
                 src_vocab_size,
                 padding_index,
                 dropout):
        
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([
            EncoderLayer(emb_dim,
                         num_heads,
                         ff_dim,
                         dropout)
            for _ in range(num_layers)
        ])
        
        self.embedding = nn.Embedding(src_vocab_size, emb_dim, padding_idx=0)
        self.pe = PositionalEncoder(emb_dim)
        
    def forward(self, src):
        src = self.embedding(src)
        
        src = self.pe(src)
        
        for layer in self.layers:
            src = layer(src, None)
            
        return src
    
class DecoderLayer(nn.Module):
    def __init__(self, emb_dim, num_heads, ff_dim, dropout):
        super(DecoderLayer, self).__init__()
        
        self.dim_kqv = emb_dim // num_heads
        
        assert (self.dim_kqv * num_heads == emb_dim), "Embedding size must be divisible by number of heads"
        
        self.attention_1 = Residual(
            MultiHeadAttention(num_heads, emb_dim, self.dim_kqv),
            dimension=emb_dim,
            dropout=dropout
        )
        
        self.attention_2 = Residual(
            MultiHeadAttention(num_heads, emb_dim, self.dim_kqv),
            dimension=emb_dim,
            dropout=dropout
        )
        
        self.feed_forward = Residual(
            FeedForward(emb_dim, ff_dim),
            dimension=emb_dim,
            dropout=dropout
        )
        
    def forward(self, trg, memory, mask):
        query = self.attention_1(trg, trg, trg, mask)
        attentions = self.attention_2(query, memory, memory, None)
        out = self.feed_forward(attentions)
        
        return out

class Decoder(nn.Module):
    def __init__(self, 
                 emb_dim, 
                 num_heads, 
                 ff_dim, 
                 num_layers, 
                 out_size, 
                 padding_index,
                 dropout):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([
            DecoderLayer(emb_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])
        
        self.embedding = nn.Embedding(out_size, emb_dim, padding_idx=padding_index)
        self.pe = PositionalEncoder(emb_dim)

    def make_trg_mask(self, trg):
        batch_size, seq_len = trg.shape[0], trg.shape[1]
        mask = torch.tril(torch.ones(batch_size, seq_len, seq_len))
        return mask
        
    def forward(self, trg, encoder_out):
        trg = self.embedding(trg)
        
        trg = self.pe(trg)

        mask = self.make_trg_mask(trg).to(trg.get_device())
        
        for layer in self.layers:
            trg = layer(trg, encoder_out, mask)
            
        # return self.lin(trg)
        return trg

class VanillaTransformer(nn.Module):
    def __init__(self, 
                 emb_dim, 
                 num_heads, 
                 ff_dim, 
                 num_layers, 
                 src_vocab_size, 
                 trg_vocab_size,
                 device,
                 padding_index,
                 dropout):
        super(VanillaTransformer, self).__init__()
        
        self.encoder = Encoder(emb_dim, 
                               num_heads, 
                               ff_dim,
                               num_layers, 
                               src_vocab_size,
                               padding_index,
                               dropout).to(device)
        
        self.decoder = Decoder(emb_dim,
                               num_heads,
                               ff_dim, 
                               num_layers,
                               trg_vocab_size,
                               padding_index,
                               dropout).to(device)

        self.lin = nn.Linear(emb_dim, trg_vocab_size)
        
        
    def forward(self, src, trg):
        encoder_out = self.encoder(src)

        decoder_out = self.decoder(trg, encoder_out)
        
        out = self.lin(decoder_out)

        return out

In [ ]:
model = VanillaTransformer(EMBEDDING_DIM = 256,
                           NUM_HEADS = 8,
                           FF_DIM = 2048,
                           NUM_LAYERS = 6,
                           SOURCE_VOCAB_SIZE = 10,
                           TARGET_VOCAB_SIZE=10,
                           device,
                           padding_index = 0,
                           DROPOUT= 0.1).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_func = nn.CrossEntropyLoss(ignore_index=padding_index)

for epoch in tqdm(range(NUM_EPOCHS), position = 0, leave = True):
    
    running_loss = 0.0
    model.train()
    
    for batch_index, batch in enumerate(train_loader):
        optimizer.zero_grad()
        
        src = batch['x_source']
        trg = batch['y_target']
        
        y_pred = model(src.to(device), trg[:, :-1].to(device))
        y_pred = y_pred.reshape(-1, y_pred.shape[2])
        
        loss = loss_func(y_pred, trg[:, 1:].reshape(-1).to(device))
        loss.backward()

        running_loss += (loss.item() - running_loss) / (batch_index + 1)
        optimizer.step()
        
    if epoch == 0 or (epoch + 1) % PRINT_EVERY == 0:
      print('Epoch: {:<2} Train loss: {:0.4f}'.format(epoch + 1 , running_loss))

In [1]:
import numpy as np

list1=[0.951,0.961,0.959,0.958,0.959,0.959,0.962]

print(np.mean(list1))
print(np.std(list1))

0.9584285714285713
0.0032888184094918135


In [2]:
import numpy as np

list2=[0.954,0.964,0.958,0.958,0.960,0.964,0.959]

print(np.mean(list2))
print(np.std(list2))

0.9595714285714284
0.003288818409491814


In [3]:
import numpy as np

list3=[0.960,0.964,0.963,0.961,0.962,0.966,0.964]

print(np.mean(list3))
print(np.std(list3))

0.9628571428571429
0.0018844151368961328


In [18]:
import numpy as np

list4=[0.969,0.956,0.965,0.952,0.967,0.957,0.957]

print(np.mean(list4))
print(np.std(list4))

0.9604285714285713
0.005996597674804393


In [23]:
import numpy as np

list5=[0.967,0.963,0.965,0.967,0.968,0.967,0.968]

print(np.mean(list5))
print(np.std(list5))

0.9664285714285714
0.0016781914463529632


In [4]:
import torch

tensor_a=torch.rand(5)
print(tensor_a)
print(tensor_a.max())
print(tensor_a.argmax())

tensor([0.4515, 0.8778, 0.8910, 0.8566, 0.7635])
tensor(0.8910)
tensor(2)


In [20]:
tensor_b=torch.rand(5)
print(tensor_b)
weight=0.5*torch.rand(1)+0.5
# weight=weight.item()
print(weight)
tensor_sum=weight*tensor_a + (1-weight)*tensor_b
print(tensor_sum)

tensor([0.7524, 0.0887, 0.6099, 0.8253, 0.1242])
tensor([0.8679])
tensor([0.4912, 0.7736, 0.8539, 0.8524, 0.6790])
